In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
from gensim.models import KeyedVectors
from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random

d:\sem8\CS772\A2_Part1\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus

In [9]:
tagged_sentences[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

In [10]:
X = [] # store input sequence
Y = [] # store output sequence

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:         
        X_sentence.append(entity[0])  # entity[0] contains the word
        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag
        
    X.append(X_sentence)
    Y.append(Y_sentence)

In [11]:
X[0]

['Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 ',',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov.',
 '29',
 '.']

In [12]:
num_words = len(set([word.lower() for sentence in X for word in sentence]))
num_tags   = len(set([word.lower() for sentence in Y for word in sentence]))
print(num_words)
print(num_tags)

59448
12


In [13]:
unique_tags = list(set([word.lower() for sentence in Y for word in sentence]))
unique_tags_dict = {}
index = 1
for tag in unique_tags:
    unique_tags_dict[tag] = index 
    index += 1
print(unique_tags_dict)

{'conj': 1, 'verb': 2, 'adj': 3, 'det': 4, 'prt': 5, 'x': 6, 'pron': 7, 'adv': 8, 'adp': 9, '.': 10, 'num': 11, 'noun': 12}


In [14]:
unique_words = list(set([word.lower() for sentence in X for word in sentence]))
unique_words_dict = {}
index = 1
for word in unique_words:
    unique_words_dict[word] = index 
    index += 1
print(len(unique_words_dict))

59448


In [15]:
values = []
for i in unique_words_dict.keys():
    values.append(unique_words_dict[i])

print(max(values))
max_value_dict = max(values)

59448


In [16]:
def prepare_sequence_sentence(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    idxs = [to_ix[w.lower()] for w in seq]
    random_index = random.randint(0,len(idxs)-1)
    idxs[random_index] = max_value_dict + 1
    return idxs

In [17]:
def prepare_sequence_tags(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    idxs = [to_ix[w.lower()] for w in seq]
    return idxs

In [18]:
from keras_preprocessing.sequence import pad_sequences
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
def split(list_a, batch_size):

  for i in range(0, len(list_a), batch_size):
    yield list_a[i:i + batch_size]

cuda


In [19]:
batch_size = 8
X_batches = list(split(X, batch_size))
Y_batches = list(split(Y,batch_size))

In [20]:
X_batches_padded = []
Y_batches_padded = []
max_length_list = []

for b_s,b_t in zip(X_batches,Y_batches):
    max_seq_length = 0
    for sentence in b_s:
        if len(sentence) > max_seq_length:
            max_seq_length = len(sentence)
    
    sen_encoded = []
    tag_encoded = []
    for sentence,tags in zip(b_s,b_t):
        sen_encoded.append(prepare_sequence_sentence(sentence, unique_words_dict))
        tag_encoded.append(prepare_sequence_tags(tags, unique_tags_dict))
    
    X_batches_padded.append(pad_sequences(sen_encoded, maxlen=max_seq_length, padding="pre", truncating="post"))
    Y_batches_padded.append(pad_sequences(tag_encoded, maxlen=max_seq_length, padding="pre", truncating="post"))
    max_length_list.append(max_seq_length)

In [21]:
print(len(Y_batches_padded))
print(len(X_batches_padded))
print(len(max_length_list))

9026
9026
9026


In [22]:
print(len(Y_batches_padded[0]))
len(X_batches_padded[0])

8


8

In [23]:
X_final = []
Y_final = []

for index in range(len(X_batches_padded)):
    X_batch_tensor = torch.zeros((batch_size,max_length_list[index]),dtype = int).to(device= device)
    Y_batch_tensor = torch.zeros((batch_size,max_length_list[index]), dtype = int).to(device = device)

    count = 0
    for x, y in zip(X_batches_padded[index],Y_batches_padded[index]):
        X_batch_tensor[count] = torch.tensor(x).to(device = device)
        Y_batch_tensor[count] = torch.tensor(y).to(device =device)
        count += 1
    
    X_final.append(X_batch_tensor)
    Y_final.append(Y_batch_tensor)

In [24]:
EMBEDDING_SIZE  = 300  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = num_words + 2

with open('./embedding_weights.pickle', 'rb') as file:
    embedding_weights = pickle.load(file)

print(embedding_weights.shape)

torch.Size([59450, 300])


In [25]:
def create_emb_layer(weights_matrix, non_trainable=False):

    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [26]:
class RNNTagger(nn.Module):
    def __init__(self, hidden_dim, target_size, batch_size,num_layers):
        super(RNNTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        
        self.word_embeddings, vocab_size, embedding_dim = create_emb_layer(embedding_weights, True)
        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first = True, num_layers = num_layers)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
        #print("REACHONG FORWARD")

        #Input shape: [batch_size,max_length in that batch]
        embeds = self.word_embeddings(sentence)
        #print("DONE embeds:", embeds.shape)  -- torch.Size([batch_Size, max_length_in_that_batch, embedding_dim])
  
        
        #input shape: [len(sentence),1,embedding_dim] (L,N,Hin​) when batch_first=False)
    
        rnn_out, hidden_state_out = self.rnn(embeds) 
        #print(rnn_out.shape)  -- torch.Size([8, 45, 64])
        #print(hidden_state_out.shape) --torch.Size([1, 8, 64])


        #input shape: -- torch.Size([8, 45, 64])
        tag_space = self.hidden2tag(rnn_out)
        #print("DONE LINEAR LAYER: ", tag_space.shape) --torch.Size([8, 45, 13])
        
        tag_scores = F.log_softmax(tag_space, dim=2)
        #print("DONE SOFTMAX:", tag_scores.shape) --torch.Size([8, 45, 13])
        
        return tag_scores

In [27]:
def train_loop(model,loss_function,optimizer,device,X,Y):
    train_length = len(X)
    epoch_train_loss = 0 
   
    model.train()
    for i in tqdm(range(train_length)):
        sentence_batch = X[i]
        tags_batch = Y[i]

        model.zero_grad()

        #print("Input shape:",  sentence_batch.shape) --torch.Size([batch_size, max_length_in_batch]) 
        tag_scores = model(sentence_batch)

       
        #print("model output shape: ", tag_scores.shape) -- torch.Size([8, 45, 13]) 
        #print("tags batch shape: ", tags_batch.shape) --torch.Size([8, 45])
        new1 = tag_scores.transpose(1,2)
        #print("new input shape:" , new1.shape) --torch.Size([8, 13, 45])

        #print("STARTING LOSS FUNCTION")
        loss = loss_function(new1 , tags_batch)
        #print("DONE WITH LOSS FUNCTION")
        #print(loss)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    return model, epoch_train_loss/train_length

In [28]:
def validation_loop(model,loss_function,device,X,Y):
    val_length = len(X)
    epoch_val_loss = 0 

    for i in tqdm(range(val_length)):
        sentence_batch = X[i]
        tags_batch = Y[i]

        model.eval()

        tag_scores = model(sentence_batch)

        #CALL A FUNCTION WITH tag_scores and targets, GET PRECISION RECALL FScores        

        new1 = tag_scores.transpose(1,2) 
        loss = loss_function(new1, tags_batch)
        epoch_val_loss += loss.item()
          
     
    
    return epoch_val_loss/val_length

In [29]:
HIDDEN_DIM = 64
batch_size = 8
num_layers =  2
model =RNNTagger(HIDDEN_DIM, len(unique_tags_dict.keys())+1, batch_size,num_layers)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device=device)

cuda


In [30]:
TEST_SIZE = 0.1
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=TEST_SIZE, random_state=4)

VALID_SIZE = 0.15
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=VALID_SIZE, random_state=4)

In [25]:
epochs = 5
for epoch in range(epochs):
    model , train_loss = train_loop(model,loss_function,optimizer,device,X_train,Y_train)
    val_loss = validation_loop(model,loss_function,device,X_validation,Y_validation)
    print("For epoch {}, training loss: {}, validation loss: {}".format(epoch, train_loss, val_loss))

100%|██████████| 1219/1219 [00:00<00:00, 1390.29it/s]


For epoch 0, training loss: 0.4518412434089139, validation loss: 0.3371057066213801


100%|██████████| 1219/1219 [00:00<00:00, 1369.09it/s]


For epoch 1, training loss: 0.3042854023978615, validation loss: 0.2888657215484544


100%|██████████| 1219/1219 [00:00<00:00, 1513.02it/s]


For epoch 2, training loss: 0.26870788583156424, validation loss: 0.264800105063616


100%|██████████| 1219/1219 [00:00<00:00, 1393.09it/s]


For epoch 3, training loss: 0.24785828086043582, validation loss: 0.2496536498900023


100%|██████████| 1219/1219 [00:00<00:00, 1274.83it/s]

For epoch 4, training loss: 0.23349588425605539, validation loss: 0.2387117083758189


In [31]:
epochs = 5
for epoch in range(epochs):
    model , train_loss = train_loop(model,loss_function,optimizer,device,X_train,Y_train)
    val_loss = validation_loop(model,loss_function,device,X_validation,Y_validation)
    print("For epoch {}, training loss: {}, validation loss: {}".format(epoch, train_loss, val_loss))

100%|██████████| 1219/1219 [00:01<00:00, 1167.23it/s]


For epoch 0, training loss: 0.5444462966737214, validation loss: 0.35320354160761813


100%|██████████| 1219/1219 [00:00<00:00, 1287.69it/s]


For epoch 1, training loss: 0.31087625902610144, validation loss: 0.2936107037715779


100%|██████████| 1219/1219 [00:00<00:00, 1314.46it/s]


For epoch 2, training loss: 0.2672839114433834, validation loss: 0.26517777010443955


100%|██████████| 1219/1219 [00:00<00:00, 1338.17it/s]


For epoch 3, training loss: 0.24279792084027327, validation loss: 0.24739950303663671


100%|██████████| 1219/1219 [00:00<00:00, 1299.82it/s]

For epoch 4, training loss: 0.22620750430038983, validation loss: 0.23453835503085899


DEMO CODE

In [ ]:
import nltk
nltk.download('punkt')

In [32]:
def prepare_TESTsequence_sentence(seq, to_ix):
    values = []
    for i in to_ix.keys():
        values.append(to_ix[i])
    
    max_value_dict = max(values)
    
    idxs = []

    for w in seq:
        if w.lower() in to_ix.keys():
            idxs.append(to_ix[w.lower()])
        else:
            idxs.append(max_value_dict+1)
    return idxs

In [33]:
index_to_tags = {}
for tag in unique_tags_dict:
    index_to_tags[unique_tags_dict[tag]] = tag 
index_to_tags[0] = '0'
index_to_tags

{1: 'conj',
 2: 'verb',
 3: 'adj',
 4: 'det',
 5: 'prt',
 6: 'x',
 7: 'pron',
 8: 'adv',
 9: 'adp',
 10: '.',
 11: 'num',
 12: 'noun',
 0: '0'}

In [34]:
index_to_tags = {}
for tag in unique_tags_dict:
    index_to_tags[unique_tags_dict[tag]] = tag 
index_to_tags[0] = '0'
index_to_tags

{1: 'conj',
 2: 'verb',
 3: 'adj',
 4: 'det',
 5: 'prt',
 6: 'x',
 7: 'pron',
 8: 'adv',
 9: 'adp',
 10: '.',
 11: 'num',
 12: 'noun',
 0: '0'}

In [35]:
from nltk.tokenize import word_tokenize
text = "bottle is 5 years old, non executive director of CSK"
tokenized_seq = word_tokenize(text)
print(tokenized_seq)

tokens= prepare_TESTsequence_sentence(tokenized_seq,unique_words_dict)
print(tokens) 

tokens = torch.tensor(tokens).to(device = device)
tokens = tokens.unsqueeze(0)
output = model(tokens)
output = output.squeeze(0)
tag_index = torch.argmax(output,dim=1)
#print(tag_index.shape)
#print(output.shape)
for predicted_tag,word in zip(tag_index,tokenized_seq):
    predicted_tag = predicted_tag.item()
    predicted_tag = index_to_tags[predicted_tag]
    print(word, predicted_tag)

['bottle', 'is', '5', 'years', 'old', ',', 'non', 'executive', 'director', 'of', 'CSK']
[53260, 45938, 42439, 1493, 49622, 25353, 2998, 34501, 34780, 7540, 59449]
bottle noun
is verb
5 num
years noun
old adj
, .
non noun
executive noun
director noun
of adp
CSK det
